In [ ]:
#deleted the subscription id for privacy since this repositry is public 
subscription_id = ''
resource_group = 'AzureQuantum'
workspace_name = 'Ruba'
location = 'Japan West'

In [2]:
import qsharp

qsharp.init(target_profile=qsharp.TargetProfile.Base)

Q# initialized with configuration: {'targetProfile': 'base', 'languageFeatures': None, 'manifest': None}

In [3]:
import qsharp
import numpy as np
import matplotlib.pyplot as plt

In [4]:
%%qsharp
open Microsoft.Quantum.Diagnostics;

In [5]:
%config dump.basisStateLabelingConvention="BitString" 
%config dump.phaseDisplayStyle="NumberOnly"

In [6]:
%%qsharp
operation RandomiseSix(target: Qubit[]) : Unit {
  ApplyToEach(H, target);    
}

In [7]:
%%qsharp
operation GateSwitch(target: Qubit[], pattern: Bool[]) : Unit {
    for i in 0..5 {
        if pattern[i] {
            X(target[i]); 
        }
    }     
}

In [8]:
%%qsharp
operation SelectorSix(target: Qubit[], pattern: Bool[]) : Unit {
    GateSwitch(target, pattern); 
    Controlled Z(target[0..4],target[5]); 
    GateSwitch(target, pattern); 
}

In [9]:
%%qsharp
operation AmplifySix(target: Qubit[]) : Unit {
    ApplyToEach(H, target); 
    ApplyToEach(X, target); 
    Controlled Z(target[0..4],target[5]);
    ApplyToEach(X, target); 
    ApplyToEach(H, target); 
}

In [10]:
%%qsharp
operation SixQGroverIteration(target: Qubit[], pattern: Bool[]) : Unit {
    SelectorSix(target, pattern); 
    AmplifySix(target); 
}

In [11]:
%%qsharp
operation SixQGrovers(target: Qubit[], pattern: Bool[], repeats: Int) : Unit {
    for i in 1..repeats {
        SixQGroverIteration(target, pattern); 
    }
}

In [12]:
%%qsharp
operation RandomiseSix(target: Qubit[]) : Unit {
  ApplyToEach(H, target);    
}

In [13]:
%%qsharp
operation SixGroverRun(repeats: Int, pattern: Bool[]) : Result[] {
    use q = Qubit[6];
    RandomiseSix(q);
    SixQGrovers(q, pattern, repeats);
    return MResetEachZ(q);        
}


In [14]:
%%qsharp

// Example: Call SixGroverRun with 2 repeats and pattern 101010:
SixGroverRun(6, [true,false,true,false,true,false])


[Zero, One, Zero, One, Zero, One]

In [15]:
import azure.quantum

workspace = azure.quantum.Workspace(
    subscription_id=subscription_id,
    resource_group=resource_group,
    name=workspace_name,
    location=location,
)

operation = qsharp.compile("SixGroverRun(6, [true,false,true,false,true,false])")

target = workspace.get_targets("quantinuum.sim.h1-1e")
job = target.submit(operation, "Grover's", shots=20)